In [1]:
%matplotlib inline

import tensorflow as tf
import math
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import os

### MNIST Dataset 가져오기

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist_data/")

train_images = mnist.train.images
train_labels = mnist.train.labels

valid_images = mnist.validation.images
valid_labels = mnist.validation.labels

test_images = mnist.test.images
test_labels = mnist.test.labels

W0728 10:37:03.979048 4470273472 deprecation.py:323] From <ipython-input-2-aebca0d60f47>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0728 10:37:03.980784 4470273472 deprecation.py:323] From /Users/ksj/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0728 10:37:03.981948 4470273472 deprecation.py:323] From /Users/ksj/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instru

Extracting mnist_data/train-images-idx3-ubyte.gz


W0728 10:37:04.207558 4470273472 deprecation.py:323] From /Users/ksj/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0728 10:37:04.254399 4470273472 deprecation.py:323] From /Users/ksj/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


### [Optional.  Tensorflow Graph Visualization ]

---

> _Jupyter에서 Tensorflow에서 구성되는 Graph를 시각적으로 보여주기 위한 helper 메소드입니다._<br>

In [3]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))

    display(HTML(iframe))

<br><br>

# \[ Conditional Variational Auto Encoder \]
---

> *Auto Encoder는 입력을 Latent Vector로 변환하는 Encoder 부분과 Latent Vectord에서 입력으로 복원하는 Decoder 부분으로 나뉘어 집니다. Variational Auto Encoder가 목표로 하는 것은 "잘 만들어진" Latent Vector을 만드는 것이 목적입니다.*

## 1. Conditional Variational AutoEncoder이란?
---

![Imgur](https://i.imgur.com/bmBYX6H.png)

Conditional Variational Auto Encoder는 어떻게 하면, 우리가 **라벨**에 따라 원하는 이미지들을 생성해낼 것인지에 대한 방법론입니다. 사실 매우 심플해요, Label 정보를 Encoder와 Decoder에 넣어주면 끝입니다.

### (1) 인코더 구성하기

![](https://i.imgur.com/uhYATU8.png)

딱 모델은 위와 같이 바뀝니다. 이게 끝입니다.

In [4]:
from tensorflow.keras.layers import Embedding, Concatenate

In [5]:
graph = tf.Graph()

num_input =784
num_label = 10
num_hidden1 = 128
num_hidden2 = 128
num_hidden3 = 2

with graph.as_default():
    x = tf.placeholder(tf.float32, shape=(None,num_input),
                       name='x')
    cond = tf.placeholder(tf.int32, shape=(None,),name='label')
    lr = tf.placeholder_with_default(0.01, None, 
                                     name='learning_rate')
    
    embeded = Embedding(10,20)(cond)
    encoder_concat = Concatenate()([x,embeded])
    encoded_1 = tf.layers.Dense(num_hidden1, 
                                activation=tf.nn.leaky_relu,
                                name='encode_1')(encoder_concat)
    encoded_2 = tf.layers.Dense(num_hidden2,
                                activation=tf.nn.relu,
                                name='encode_2')(encoded_1)   
    
    with tf.variable_scope("reparameterization_trick"):
        # Encoder 중 확률분포의 평균을 찾는 Layer
        code_mean = tf.layers.Dense(num_hidden3,
                                   name='code_mean')(encoded_2)
        
        # Encoder 중 확률분포의 분산을 찾는 Layer
        code_sigma = tf.layers.Dense(num_hidden3,
                                   activation=tf.nn.softplus,
                                   name='code_gamma')(encoded_2)
        
        # reparameterization trick
        # 아래와 같이 확률분포에서 Sampling하게 되면서, 
        # code_mean과 code_sigma 쪽으로 역전파가 가능하게 됨
        noise = tf.random.normal(tf.shape(code_sigma))
        sampled_code = code_mean + code_sigma * noise
    sampled_code = tf.identity(sampled_code,"sampled_code")    

W0728 10:37:04.586533 4470273472 deprecation.py:506] From /Users/ksj/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 10:37:04.606394 4470273472 deprecation.py:506] From /Users/ksj/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
show_graph(graph)

### (3) Decoder 구성하기

여느 Stacked Auto Encoder와 같이 복원하는 방향으로 학습하게 됩니다.

In [7]:
with graph.as_default():
    decoder_concat = Concatenate()([x,embeded])
    decoded_2 = tf.layers.Dense(num_hidden2, 
                                activation=tf.nn.leaky_relu,
                                name='decode_2')(decoder_concat)
    decoded_1 = tf.layers.Dense(num_hidden1,
                                activation=tf.nn.leaky_relu,
                                name='decode_1')(decoded_2)    
    logits = tf.layers.Dense(num_input,
                              name='logits')(decoded_1)
    outputs = tf.sigmoid(logits, name='outputs')

In [8]:
show_graph(graph)

### (4) Loss 구성하기

Loss는 Conditional Auto Encoder와 동일

In [9]:
epsilon = 1e-8
with graph.as_default():        
    with tf.variable_scope("losses"):
        reconstruction_loss = -tf.reduce_sum(x * tf.log(epsilon+outputs) + 
                                    (1-x) * tf.log(epsilon+1-outputs),1)
        reconstruction_loss = tf.reduce_mean(reconstruction_loss)
        with tf.variable_scope("kl_divergence"):
            latent_loss = 0.5 * tf.reduce_sum(
                tf.square(code_mean) + tf.square(code_sigma) - 
                tf.log(epsilon + tf.square(code_sigma)) - 1,1)
            latent_loss = tf.reduce_mean(latent_loss)
        loss = reconstruction_loss + latent_loss

In [10]:
show_graph(graph)

### (5) 학습을 위한 operation 구성하기

잘 복원했는지에 대한 기준으로는 Root-mean-squared-error을 이용할 예정이고, Adam Optimizer을 이용해 모델을 학습시켰습니다.

In [11]:
with graph.as_default():
    with tf.variable_scope('metric'):
        mse = tf.losses.mean_squared_error(x, outputs)
        rmse = tf.sqrt(mse, name='rmse')
        
    with tf.variable_scope('train'):
        train_op = (tf.train
                    .AdamOptimizer(lr)
                    .minimize(loss, name='train_op'))

W0728 10:37:06.025547 4470273472 deprecation.py:323] From /Users/ksj/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
show_graph(graph)

### (6) 모델 학습시키기

In [ ]:
num_epoch = 100 # epoch 횟수
num_batch = 128 # 배치 크기
num_data = mnist.train.num_examples # data의 수
num_step = num_data // num_batch # 1 epoch 별 학습 횟수

with graph.as_default():
    sess = tf.Session(graph=graph)
    
    sess.run(tf.global_variables_initializer())
    for i in range(num_epoch):        
        for _ in range(num_step):
            images, labels = mnist.train.next_batch(num_batch)
            sess.run(train_op, feed_dict={x:images,cond:labels,lr:0.001})
        
        rec, lat, tot, rmse_value = sess.run(
            [reconstruction_loss,latent_loss, loss, rmse], 
            feed_dict={x:valid_images,cond:valid_labels})
        print("{:2d}th epoch 전체 손실 : {:.3f}, 재구성 손실 : {:.3f} 잠재 손실 : {:.5f} RMSE : {:.3f}"
              .format(i,tot,rec,lat, rmse_value))

In [ ]:
ys,xs = np.mgrid[1:-1:-0.1,-1:1:0.1]
points = np.stack([xs.ravel(),ys.ravel()],axis=-1)

grid_images = sess.run(outputs, 
                       feed_dict={sampled_code:points})

plt.figure(figsize=(10,10))
plt.imshow(grid_images
           .reshape(20,20,28,28)
           .transpose(0,2,1,3)
           .reshape(560,560))
plt.show()

In [ ]:
plt.imshow(grid_images.reshape(-1,28,28)[9])

### 참고 자료들
1. [Variational Auto Encoder: Intuition and Implementation](https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/)
2. [알기쉬운 Variational autoencoder](https://www.slideshare.net/ssuser06e0c5/variational-autoencoder-76552518)
3. [Auto Encoder의 모든 것](https://www.slideshare.net/NaverEngineering/ss-96581209)
4. [초짜 대학원생의 입장에서 이해하는 Auto-Encoding Variational Bayes (VAE) (2)](http://jaejunyoo.blogspot.com/2017/04/auto-encoding-variational-bayes-vae-2.html)
5. [A Briefl Introduction to Manifold Learning](https://www.slideshare.net/plutoyang/manifold-learning-64891420)